In [7]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# If working in Colab, mount your drive
from google.colab import drive
drive.mount('/content/drive')

# Direct to your project folder.
%cd /content/drive/Othercomputers/Computer1/dl-robocop

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# Installing requirements
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
#from datasets import load_dataset
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
#import evaluate
#import gensim
import transformers
import nltk
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data.dataset import random_split
#import jsonlines

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# BART MODEL (STUDENT)
model_name = "facebook/bart-large"
student_tokenizer = AutoTokenizer.from_pretrained(model_name)
student_tokenizer.pad_token_id = student_tokenizer.eos_token_id
student_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

#model.load_state_dict(torch.load('bart_test.pth'))

In [ ]:
# PARADETOX MODEL (TEACHER)
#from transformers import BartForConditionalGeneration, AutoTokenizer
#base_model_name = 'facebook/bart-base'
#model_name = 'SkolkovoInstitute/bart-base-detox'
#teacher_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
#teacher_model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

In [10]:
# POLITENESS JUDGE
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# load tokenizer and model weights
classifier_tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
toxicity_clasifier_model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

C:\Users\41786\OneDrive - epfl.ch\master\ma2\dl-robocop\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\41786\.cache\huggingface\hub\models--SkolkovoInstitute--roberta_toxicity_classifier. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
from helpers import format_data

data_path = 'data/'
dataset = pd.read_csv(data_path + 'paradetox.tsv', sep='\t')

# Define the maximum length and other parameters
max_length = 38
tensor_dataset = format_data(max_length, student_tokenizer, dataset)

# Define the size of the training set
train_size = int(0.8 * len(tensor_dataset))
test_size = len(tensor_dataset) - train_size

# Split the dataset into training and test sets
train_dataset, test_dataset = random_split(tensor_dataset, [train_size, test_size])

# Defining parameters for training
batch_size = 16
lr = 3e-5
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = 16, shuffle = False)
optimizer = torch.optim.Adam(student_model.parameters(), lr=lr)
num_epochs = 1

In [49]:
from helpers import evaluate, calculate_toxicity_score, policy_sampling, train_on_paradetox
train_on_paradetox(student_model = student_model,
                   judge_model = toxicity_clasifier_model,
                   tokenizer = student_tokenizer,
                   judge_tokenizer = classifier_tokenizer,
                   optimizer =optimizer,
                   dataloader = train_dataloader,
                   test_dataloader = test_dataloader,
                   num_epochs = num_epochs,
                   alpha = 0.5,
                   device = device,
                   lambda_tox = 1,
                   lambda_bert = 1)


 1/1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Average train Loss: 5.9861
Evaluating on test set...
Test Average Loss: 3.5201


In [60]:
# TODO: delete this cell (only for testing)
# Load .pt file of the trained model
trained_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
trained_model.load_state_dict(torch.load('weights/epoch0.pt'))

for batch in test_dataloader:
  input_ids, detoxified_ids, input_mask = batch

  # Move tensors to the device
  input_ids = input_ids.to(device)
  detoxified_ids = detoxified_ids.to(device)
  input_mask = input_mask.to(device)

  # Disable gradient computation
  with torch.no_grad():
      # Forward pass
      model_outputs = trained_model(input_ids=input_ids, attention_mask=input_mask, labels=detoxified_ids)
      # Print the text input
      print('Toxic sentence: ', ' '.join([student_tokenizer.decode(i, skip_special_tokens = True) for i in input_ids[0]]))
      # Print the detoxified output
      print('Detoxified sentence: ', ' '.join([student_tokenizer.decode(i, skip_special_tokens = True) for i in model_outputs.logits.argmax(dim=-1)[0]]))

Toxic sentence:   he  had  steel  balls  too !                               
Detoxified sentence:                                       
tensor([[[-11.9701,  -3.1708,  27.6744,  ...,   1.2927,  -1.6379,  18.4402],
         [-17.5852,  -3.7942,  28.5393,  ...,   0.8406,  -2.1964,  13.7490],
         [-13.8262,  -3.6494,  31.1646,  ...,   0.6308,  -1.5011,  16.7933],
         ...,
         [ -4.3621,  -3.5353,  30.5061,  ...,  -1.1026,  -3.2969,  18.2109],
         [-17.6482,  -3.9246,  31.9844,  ...,  -2.0402,  -3.3423,  13.9271],
         [ -4.1738,  -3.5651,  30.5883,  ...,  -1.0210,  -3.3321,  17.9093]]])


IndexError: tuple index out of range

In [ ]:
prompt = "Rewrite the following toxic input into non-toxic version. Let's break the input down step by step to rewrite the non-toxic version. You should first think about the expanation of why the input text is toxic. Then generate the detoxic output. You must preserve the original meaning as much as possible.\nInput: "

for (idx,batch) in enumerate(test_dataloader):
  #input = prompt+input+"\n"
  model.eval()
  input_ids, detoxified_ids, mask = batch

  input_ids = input_ids.to(device)
  detoxified_ids = detoxified_ids.to(device)
  mask = mask.to(device)

  outputs = model.generate(input_ids = input_ids, attention_mask = mask, max_new_tokens = 50, do_sample=False)
  print('Toxic sentence: ', ' '.join([tokenizer.decode(i, skip_special_tokens = True) for i in input_ids[0]]))
  print(f'Detoxified sentence:\n {tokenizer.decode(outputs[0], skip_special_tokens = True)}')
  print('Gold: ', ' '.join([tokenizer.decode(i, skip_special_tokens = True) for i in detoxified_ids[0]]))
  print()
  if idx == 10:
    break